<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">Thierry Parmentelat &amp; Arnaud Legout&nbsp;<img src="media/both-logos-small-alpha.png" style="display:inline"></span><br/>

In [ ]:
import asyncio

from asynchelpers import start_timer, show_timer

In [ ]:
async def countdown(ticks, quotient, period=1):
    """
    affiche un compte à rebours de ticks periodes
    puis retourne 1/quotient
    """
    n = ticks
    while n > 0:
        show_timer(f"tick{n}({quotient})")
        n -= 1
        await asyncio.sleep(period)
    show_timer(f"countdown ({ticks} x {period}) done")
    return 1 / quotient

In [ ]:
async def monitor_tasks(tasks, period):
    """
    Affiche toutes les <period> un état de toutes
    les taches dans tasks, 
    et retourne lorsqu'elles sont toutes terminées
    """
    while True:
        message = "monitor:"
        alive = 0
        for i, task in enumerate(tasks):
            message += f" {i} -> {task.done()}"
            if not task.done():
                alive += 1
        # s'il n'en reste qu'une c'est le monitor
        show_timer(message)
        if alive == 0:
            show_timer("exiting")
            return
        await asyncio.sleep(period)

In [ ]:
tasks = [
    # va terminer normalement
    asyncio.ensure_future(countdown(2, quotient=1)),
    # lève une exception
    asyncio.ensure_future(countdown(3, quotient=0)),
]

In [ ]:
start_timer()
asyncio.get_event_loop().run_until_complete(
    monitor_tasks(tasks, .8)
)

In [ ]:
show_timer("cleaning up")
for i, task in enumerate(tasks):
    if not task.done():
        # monitor_task should exit only when all tasks are done
        show_timer("OOPS, this is unexpected")
    elif task.exception():
        show_timer(f"task: {i} has raised exception {task.exception()}")
    else:
        show_timer(f"task: {i} has returned {task.result()}")